# Python - Logging

- [Understanding pythons logging module (don't log directly against the root logger)](https://www.electricmonk.nl/log/2017/08/06/understanding-pythons-logging-module/)
- [System Development with Python - Logging and the logging module](http://uwpce-pythoncert.github.io/SystemDevelopment/logging.html)
- [python logging配置時間或大小輪轉](http://www.361way.com/python-logging-timedrotating/5043.html)
- [Python 3 Logging using DictConfig](https://simpletutorials.com/c/1457/Python+3+Logging+using+DictConfig)
- [Logging Exceptions](https://stackoverflow.com/questions/5191830/how-do-i-log-a-python-error-with-debug-information)
- [Python Logging: An In-Depth Tutorial](https://www.toptal.com/python/in-depth-python-logging)
- [StackOverflow - Using Python logging in multiple modules
](https://stackoverflow.com/questions/15727420/using-python-logging-in-multiple-modules)

## Basic Concept

### The logging classes

http://uwpce-pythoncert.github.io/SystemDevelopment/logging.html

- **Loggers** - the interface for your code
- **Handlers** - handle log routing
- **Filters** - define which log messages to let through. It uses the logger hierarchy from `logging.getLogger("A.B.C")`
- **Formatters** - how the log messages get rendered

![python-logging](./img/python-logging.png)

### Formatters

```python
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
```

Common LogRecord attributes:

```
%(asctime)s    current time. default format: '2003-07-08 16:49:45,896'
%(name)s       logger name
%(levelname)s  the "level" of the log message: debug, warning, etc.
%(filename)s   the filename of the message came from
%(module)s     the name of the module the message came from
%(message)s    the message itself
```

There are lots of other options:

https://docs.python.org/3/library/logging.html#logrecord-attributes

### Logging "levels"

When log level is `0`, it look **up** the hierarchy for the first logger with a **non-zero level**.

The built-in set is:

```
CRITICAL    50
ERROR       40
WARNING     30
INFO        20
DEBUG       10
```

Levels could be set in `Handlers` and `Loggers`. When the message is fired, it will be passed to relevant `Handlers` and `Loggers` and see if it passes the log level.

```python
my_logger.setLevel(logging.DEBUG) # or just use the arugument as string 'DEBUG'
console_handler.setLevel(logging.DEBUG)
```

### Filters - Logging hierarchy

A good convention to use when naming loggers is to use a **module-level logger**, in each module which uses logging, named as follows:

```python
logger = logging.getLogger(__name__)
```

Multiple calls to [`getLogger()`](https://docs.python.org/3.6/library/logging.html#logging.getLogger "logging.getLogger") with the same name will return a **reference to the same logger object**. Loggers that are further down in the hierarchical list are children of loggers higher up in the list. For example, given a logger with a name of `foo`, loggers with names of`foo.bar`, `foo.bar.baz`, and `foo.bam` are all descendants of `foo`.


**Example:**

在Handler上添加了一個過濾器，現在我們輸出日誌信息的時候就會經過過濾器的處理。名為 `"chat.gui"` 的過濾器只讓名字帶有 `"chat.gui"` 前綴的 Logger 輸出信息。可以添加多個過濾器，只要有一個過濾器拒絕，日誌信息就不會被輸出。另外，在Logger中也可以添加過濾器。


```python
filter_chat_gui = logging.Filter("chat.gui")
handler_console = logging.StreamHandler()

logger=logging.getLogger("chat.gui.statistic")
logger.addHandler(handler_console)
h_console.addFilter(filter_chat_gui)
```

---

## TD;DR

- All loggers are descendants of the `root logger`.
- Don't log directly against the `root logger`. That means: no `logging.basicConfig()` or `logging.getLogger()` and no usage of module-level loggers such as `logging.warning()`, as they have unintended side-effects (no output, or double lines, etc.). (see [Appendix](#Appendix))
- Configure a specific "main" logger for your program and put all the other loggers under that logger.
- The easiest way that's usually correct is to use `__name__` as the logger name: `logger = logging.getLogger(__name__)`. This uses the module hierarchy as the name, which is generally what you want. 

## Best practices

- Configure the root logger but never use it in your code --- e.g., never call a function like `logging.info()`, which actually calls the root logger behind the scene. If you want to catch error messages from libraries you use, make sure to configure the root logger to write to a file, for example, to make the debugging easier. By default, the root logger only outputs to `stderr`, so the log can get lost easily.
- To use the logging, make sure to create a new logger by using `logging.getLogger(logger name)`. I usually use  `__name__` as the logger name, but anything can be used, as long as it is consistent. To add more handlers, I usually have a method that returns a logger (you can find the gist [here](https://gist.github.com/nguyenkims/e92df0f8bd49973f0c94bddf36ed7fd0)).

In general, __your program or library shouldn’t log directly against the root logger__. Instead configure a specific **"main" logger** for your program and put all the other loggers under that logger. This way, you can toggle logging for your specific program on and off by setting the level of the main logger.

```
Project 
├── __init__.py
├── logging.conf
├── pkg01
│   └── test01.py
└── pkg02
    └── test02.py
```

Best practice is

1. In your main program or `__init__.py` file, do:

```py
# __init__.py
def main():
    """your program code"""

if __name__ == '__main__':
    import logging.config
    logging.config.fileConfig('/path/to/logging.conf')
    main()
```

2. In each module, to have a logger defined near the top of the module like this:

```py
# test01.py
import logging
logger = logging.getLogger(__name__)

# and then in other code in the module do e.g.
logger.debug('My message with %s', 'variable data')
```

## Get Started

**Use Custom logger**

```python
import logging

# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to console_handler
console_handler.setFormatter(formatter)

# add console_handler to logger
logger.addHandler(console_handler)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warn('warn message')
logger.error('error message')
logger.critical('critical message')
```


**Use Root logger (basicConfig)**

> Just for demonstration. Never use root logger!!!

It's not immediately obvious, but what this does is configure the **root logger**. Doing this may cause some counter-intuitive behaviour, because it causes debugging output for **all loggers** in your program, including every library that uses logging. This is why the requests module suddenly starts outputting debug information when you configure the root logger.

```python
import logging

logging.basicConfig(
    level=logging.DEBUG,  
    format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    filename='/tmp/test.log',
    filemode='w'  # default "a" for append; "w" for write.
)

logging.debug('debug message')  
logging.info('info message')  
logging.warning('warning message')  
logging.error('error message')  
logging.critical('critical message')  
```

The result:

```
Wed, 07 Nov 2018 23:33:19 <ipython-input-1-45bdb6a32ce8>[line:11] DEBUG debug message
Wed, 07 Nov 2018 23:33:19 <ipython-input-1-45bdb6a32ce8>[line:12] INFO info message
Wed, 07 Nov 2018 23:33:19 <ipython-input-1-45bdb6a32ce8>[line:13] WARNING warning message
Wed, 07 Nov 2018 23:33:19 <ipython-input-1-45bdb6a32ce8>[line:14] ERROR error message
Wed, 07 Nov 2018 23:33:19 <ipython-input-1-45bdb6a32ce8>[line:15] CRITICAL critical message
```

What does basicConfig do for you?

```python
filename = '/tmp/test.log'
filemode = 'w'
handler = logger.FileHandler(filename, mode)
format_str = '%(asctime)s %(message)s'
fmt = logger.Formatter(format_str)
handler.setFormatter(fmt)
logging.root.addHandler(h)
logging.root.setLevel(logging.DEBUG)
```

---

## FAQ

### How Logging Exceptions?

Setting argument `exc_info=True` will output a stack trace alongside the error message.

For example:

In [1]:
import logging
try:
    1/0
except Exception as e:
    logging.error("message", exc_info=True)

ERROR:root:message
Traceback (most recent call last):
  File "<ipython-input-1-a7f0e7b79c7b>", line 3, in <module>
    1/0
ZeroDivisionError: division by zero


In [2]:
import logging
try:
    1/0
except Exception as e:
    logging.exception("message")

ERROR:root:message
Traceback (most recent call last):
  File "<ipython-input-2-617bab5d92d6>", line 3, in <module>
    1/0
ZeroDivisionError: division by zero


### Debugging logging problems

When I run into weird logging problems such as **no output**, or **double lines**, I generally put the following debugging code at the point where I'm logging the message.

```py
log_to_debug = logging.getLogger("myapp.ui.edit")
while log_to_debug is not None:
    print(f'level: {log_to_debug.level}, name: {log_to_debug.name}, handlers: {log_to_debug.handlers}')
    log_to_debug = log_to_debug.parent
```

which outputs:

```
level: 0, name: myapp.ui.edit, handlers: []
level: 0, name: myapp.ui, handlers: []
level: 0, name: myapp, handlers: []
level: 30, name: root, handlers: []
```

---

## Appendix

### Test for logging hierarchy

In [3]:
import logging

# 創建一個logger  
logger = logging.getLogger()

logger1 = logging.getLogger('mylogger')
logger1.setLevel(logging.DEBUG)

logger2 = logging.getLogger('mylogger')
logger2.setLevel(logging.INFO)

logger3 = logging.getLogger('mylogger.child1')
logger3.setLevel(logging.WARNING)

logger4 = logging.getLogger('mylogger.child1.child2')
logger4.setLevel(logging.DEBUG)

logger5 = logging.getLogger('mylogger.child1.child2.child3')
logger5.setLevel(logging.DEBUG)

# 創建一個handler，用於寫入日誌文件  
fh = logging.FileHandler('/tmp/test.log')

# 再創建一個handler，用於輸出到控制台  
ch = logging.StreamHandler()

# 定義handler的輸出格式formatter  
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)

#定義一個filter
log_filter = logging.Filter('mylogger.child1.child2')
#fh.addFilter(log_filter)  

# 給logger添加handler  
#logger.addFilter(log_filter)
logger.addHandler(fh)
logger.addHandler(ch)

#logger1.addFilter(log_filter)
logger1.addHandler(fh)
logger1.addHandler(ch)

logger2.addHandler(fh)
logger2.addHandler(ch)

#logger3.addFilter(log_filter)
logger3.addHandler(fh)
logger3.addHandler(ch)

#logger4.addFilter(log_filter)
logger4.addHandler(fh)
logger4.addHandler(ch)

logger5.addHandler(fh)
logger5.addHandler(ch)

# 記錄一條日誌  
logger.debug('logger debug message')
logger.info('logger info message')
logger.warning('logger warning message')
logger.error('logger error message')
logger.critical('logger critical message')

logger1.debug('logger1 debug message')
logger1.info('logger1 info message')
logger1.warning('logger1 warning message')
logger1.error('logger1 error message')
logger1.critical('logger1 critical message')

logger2.debug('logger2 debug message')
logger2.info('logger2 info message')
logger2.warning('logger2 warning message')
logger2.error('logger2 error message')
logger2.critical('logger2 critical message')

logger3.debug('logger3 debug message')
logger3.info('logger3 info message')
logger3.warning('logger3 warning message')
logger3.error('logger3 error message')
logger3.critical('logger3 critical message')

logger4.debug('logger4 debug message')
logger4.info('logger4 info message')
logger4.warning('logger4 warning message')
logger4.error('logger4 error message')
logger4.critical('logger4 critical message')

logger5.debug('logger5 debug message')
logger5.info('logger5 info message')
logger5.warning('logger5 warning message')
logger5.error('logger5 error message')
logger5.critical('logger5 critical message')

2019-07-13 14:04:45,059 - root - WARNING - logger warning message
ERROR:root:logger error message
2019-07-13 14:04:45,064 - root - ERROR - logger error message
CRITICAL:root:logger critical message
2019-07-13 14:04:45,072 - root - CRITICAL - logger critical message
2019-07-13 14:04:45,080 - mylogger - INFO - logger1 info message
INFO:mylogger:logger1 info message
2019-07-13 14:04:45,080 - mylogger - INFO - logger1 info message
2019-07-13 14:04:45,098 - mylogger - WARNING - logger1 warning message
2019-07-13 14:04:45,098 - mylogger - WARNING - logger1 warning message
2019-07-13 14:04:45,112 - mylogger - ERROR - logger1 error message
ERROR:mylogger:logger1 error message
2019-07-13 14:04:45,112 - mylogger - ERROR - logger1 error message
2019-07-13 14:04:45,132 - mylogger - CRITICAL - logger1 critical message
CRITICAL:mylogger:logger1 critical message
2019-07-13 14:04:45,132 - mylogger - CRITICAL - logger1 critical message
2019-07-13 14:04:45,144 - mylogger - INFO - logger2 info message
IN

### Logging flow - from python doc

![logging_flow](https://docs.python.org/3/_images/logging_flow.png)

### Dict logger settings

```python
# logging_config.py

config = {
    'version': 1,  # int representing the schema version. The only valid value at present is 1
    'formatters': {
        'default': {
            'format': '%(asctime)s - %(levelname)s - %(message)s', 
            'datefmt': '%Y-%m-%d %H:%M:%S'
        }
    },
    'handlers': {
        'console': {
            'level': 'DEBUG',  # default 'WARNING'
            'class': 'logging.StreamHandler',
            'formatter': 'default',
            'stream': 'ext://sys.stdout'
        },
        'file': {
            'level': 'DEBUG',  # default 'WARNING'
            'class': 'logging.handlers.RotatingFileHandler',
            'formatter': 'default',
            'filename': log_path,
            'maxBytes': 1024,
            'backupCount': 3
        }
    },
    'loggers': {
        'default': {
            'level': level,  # default 'WARNING'
            'handlers': ['console', 'file']
        }
    },
    'disable_existing_loggers': False
}
```

```python
import logging
import logging.config
from logging_config import config


def configure_logger(name, log_path, level):
    logging.config.dictConfig(config)
    return logging.getLogger(name)

logger = configure_logger('default', 'log13.txt', 'DEBUG')
```

## Find the root Logger

```python
In [1]: import logging

In [2]: lab = logging.getLogger("a.b")

In [3]: lab.parent
Out[3]: <RootLogger root (WARNING)>

In [4]: lab.parent.name
Out[4]: 'root'

In [5]: la = logging.getLogger("a")

In [6]: lab.parent.name
Out[6]: 'a'
 
In [7]: root_logger = logging.getLogger()

In [8]: root_logger.name
Out[8]: 'root'

In [9]: root_logger.handlers[0]
Out[9]: <StreamHandler <stderr> (NOTSET)>
```